In [ ]:
# clean last run result

!('./clean.sh')

In [ ]:
from thunder_ase.fireball import Fireball
import numpy as np
import ase
from ase import units
from ase.md.nvtberendsen import NVTBerendsen
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

## Basic Setup

In [ ]:
# Settings for the size of the system
max_step = 100  # step for MD
cell_min = 10  # size of the cell, in angstrom

In [ ]:
# Read Cif structure

atoms = ase.io.read('Ga2O3_monoclinic.cif')

In [ ]:
super_matrix = np.eye(3) * [int(cell_min / i)+1 for i in atoms.cell.lengths()]

In [ ]:
atoms = ase.build.make_supercell(atoms, super_matrix)
#atoms.pbc = False  # make it as cluster

In [ ]:
# set Fdata dir
Fdata_path = '/home/ren/Fdata/Fdata-McWEDA-0.15-3SN.Os3.35p3.80-3SNP.Gas4.85p5.60d5.60.Ins5.45p6.20d6.20'

In [ ]:
kwargs = {
          'ipi': 1,  # open I-PI socket to speed up
          'efermi_T': 500.0,  # for metal cluster need larger Fermi smear
          }

In [ ]:
fireball = Fireball(command='fireball-ase.3.x', 
                Fdata_path=Fdata_path,
                **kwargs)
atoms.calc = fireball

In [ ]:
MaxwellBoltzmannDistribution(atoms, temperature_K=800)
dyn = NVTBerendsen(atoms, timestep=1.0 * units.fs, temperature_K=800,
                   trajectory='md-nvt.traj', logfile='md-nvt.log', 
                   taut=0.5*1000*units.fs)

fireball.dynamics(dyn, steps=max_step)